In [ ]:
"""
Шаблон для решения задач Computer Vision с использованием открытых моделей.
Этот шаблон включает:
- Загрузку и препроцессинг изображений
- Использование предобученных моделей из Hugging Face (например, для классификации, детекции объектов, сегментации)
- Обучение/дообучение (fine-tuning) модели
- Сохранение и загрузку модели
- Простую визуализацию результатов
"""

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from datasets import Dataset as HFDataset # pip install datasets
from transformers import (AutoImageProcessor, AutoModelForImageClassification,
                          AutoModelForObjectDetection, AutoModelForSemanticSegmentation,
                          TrainingArguments, Trainer)
import evaluate # pip install evaluate
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import os
from pathlib import Path

# 1. --- Загрузка и препроцессинг изображений ---
class ImageDataset(Dataset):
    """
    Пример кастомного датасета PyTorch.
    Адаптируйте под свой формат данных (пути к файлам, лейблы).
    """
    def __init__(self, image_paths, labels, processor, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        # Препроцессинг через image_processor (например, resize, normalize)
        inputs = self.processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze() # Убираем лишнюю размерность
        return {"pixel_values": pixel_values, "labels": label}

# Альтернатива: использование HuggingFace datasets
def load_hf_dataset(image_dir, labels_file_or_dict):
    """
    Загрузка датасета через HuggingFace datasets.
    image_dir: путь к папке с изображениями
    labels_file_or_dict: список или словарь с лейблами
    """
    image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]
    labels = labels_file_or_dict # Адаптируйте под свой формат

    def gen():
        for img_path, label in zip(image_paths, labels):
            yield {"image": Image.open(img_path).convert("RGB"), "label": label}

    hf_dataset = HFDataset.from_generator(gen)
    return hf_dataset

# 2. --- Примеры моделей из Hugging Face ---
# Замените на конкретные модели под вашу задачу
MODEL_NAME_CLASSIFICATION = "google/vit-base-patch16-224" # Пример: ViT для классификации
MODEL_NAME_OBJECT_DETECTION = "facebook/detr-resnet-50" # Пример: DETR для детекции
MODEL_NAME_SEGMENTATION = "nvidia/segformer-b0-finetuned-ade-512-512" # Пример: SegFormer для сегментации

TASK_TYPE = "classification" # "classification", "object_detection", "segmentation"

def get_model_and_processor(model_name, task_type):
    """
    Загружает модель и препроцессор из Hugging Face.
    """
    image_processor = AutoImageProcessor.from_pretrained(model_name)
    if task_type == "classification":
        model = AutoModelForImageClassification.from_pretrained(model_name)
    elif task_type == "object_detection":
        model = AutoModelForObjectDetection.from_pretrained(model_name)
    elif task_type == "segmentation":
        model = AutoModelForSemanticSegmentation.from_pretrained(model_name)
    else:
        raise ValueError(f"Unsupported task_type: {task_type}")
    return model, image_processor

# 3. --- Основная функция ---
def main():
    # --- Настройки ---
    model_name = MODEL_NAME_CLASSIFICATION # Выберите нужную модель
    task_type = TASK_TYPE
    image_dir = "path/to/your/images" # Путь к папке с изображениями
    labels_file_or_dict = [0, 1, 0, 1, ...] # Адаптируйте под ваш датасет
    output_dir = "./results"
    num_labels = 2 # Количество классов для классификации

    # --- Загрузка модели и препроцессора ---
    model, image_processor = get_model_and_processor(model_name, task_type)

    # --- Загрузка датасета ---
    # Вариант 1: через HuggingFace Dataset
    hf_dataset = load_hf_dataset(image_dir, labels_file_or_dict)

    # Препроцессинг датасета для конкретной задачи
    def preprocess(examples):
        inputs = image_processor(examples["image"], return_tensors="pt")
        # Для классификации
        inputs["labels"] = examples["label"]
        # Для детекции/сегментации могут потребоваться дополнительные поля
        return inputs

    processed_dataset = hf_dataset.with_transform(preprocess)

    # --- Подготовка метрик ---
    metric = evaluate.load("accuracy") # Для классификации

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return metric.compute(predictions=predictions, references=labels)

    # --- Обучение ---
    training_args = TrainingArguments(
        output_dir=output_dir,
        remove_unused_columns=False, # Важно для HuggingFace Transformers
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir=f"{output_dir}/logs",
        logging_steps=10,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        warmup_steps=500,
        weight_decay=0.01,
        report_to=None, # Отключить логгирование в wandb/mlflow
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=processed_dataset, # Разделите на train/val
        eval_dataset=processed_dataset, # Используйте отдельный валидационный сет
        tokenizer=image_processor, # Иногда требуется
        compute_metrics=compute_metrics,
    )

    # trainer.train() # Раскомментируйте для запуска обучения

    # --- Сохранение модели ---
    # trainer.save_model(f"{output_dir}/fine_tuned_model")

    # --- Инференс ---
    # Загрузка сохраненной модели (или используйте `model` после тренировки)
    # loaded_model = AutoModelForImageClassification.from_pretrained(f"{output_dir}/fine_tuned_model")
    # loaded_processor = AutoImageProcessor.from_pretrained(f"{output_dir}/fine_tuned_model")

    # Пример инференса
    example_image_path = "path/to/your/example_image.jpg"
    example_image = Image.open(example_image_path).convert("RGB")

    inputs = image_processor(example_image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_idx = logits.argmax(-1).item()
        # print("Predicted class:", model.config.id2label[predicted_class_idx])

    # --- Визуализация (для классификации - просто отображение изображения) ---
    plt.imshow(example_image)
    plt.title(f"Predicted: {model.config.id2label[predicted_class_idx]}")
    plt.axis('off')
    plt.show()

if __name__ == "__main__":
    main()